In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-sju-aichallenge-4-others/train.csv
/kaggle/input/2022-sju-aichallenge-4-others/test.csv
/kaggle/input/2022-sju-aichallenge-4-others/submit_sample.csv


In [22]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

In [23]:
train=pd.read_csv("/kaggle/input/2022-sju-aichallenge-4-others/train.csv")
x_test=pd.read_csv("/kaggle/input/2022-sju-aichallenge-4-others/test.csv")

In [24]:
train.columns

Index(['index', 'Unnamed: 0', 'customerID', 'gender', 'SeniorCitizen',
       'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges',
       'Churn'],
      dtype='object')

In [391]:
x_train=train.drop(['index','Unnamed: 0','customerID','Churn','TotalCharges'],axis=1)
y_train=pd.DataFrame(train['Churn'],index=train['Churn'].index,columns=['Churn'])
x_test=x_test.drop(['index','Unnamed: 0','customerID','TotalCharges'],axis=1)

In [392]:
ps=['SeniorCitizen','tenure','MonthlyCharges']

In [26]:
cat_cols=x_train.columns.tolist()

In [395]:
for c in cat_cols:
    if(c in ps):
        continue
    else:
        le=LabelEncoder()
        le.fit(x_train[c])
        x_train[c]=le.transform(x_train[c])
        x_test[c]=le.transform(x_test[c])

In [28]:
le_y=LabelEncoder()
ft=le_y.fit_transform(y_train)
y_train=pd.DataFrame(ft,index=y_train.index,columns=y_train.columns)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [402]:
scores_acc=[]

kf=KFold(n_splits=4,shuffle=True,random_state=77)
for tr_idx,va_idx in kf.split(x_train):
    tr_x,va_x=x_train.iloc[tr_idx],x_train.iloc[va_idx]
    tr_y,va_y=y_train.iloc[tr_idx],y_train.iloc[va_idx]
    model = xgb.XGBClassifier(learning_rate=0.291,n_estimators=100,random_state=77,tree_method='gpu_hist',gamma=10)

    %time model.fit(tr_x,tr_y)
    
    va_pred=model.predict_proba(va_x)[:,1]
    acc=accuracy_score(va_y,va_pred>0.5)
    scores_acc.append(acc)
loss=np.mean(scores_acc)
print(loss)

CPU times: user 74.5 ms, sys: 3.97 ms, total: 78.4 ms
Wall time: 57.6 ms
CPU times: user 75.6 ms, sys: 3.03 ms, total: 78.6 ms
Wall time: 56.3 ms
CPU times: user 75.8 ms, sys: 2.96 ms, total: 78.8 ms
Wall time: 56.2 ms
CPU times: user 78.1 ms, sys: 3 ms, total: 81.1 ms
Wall time: 56.6 ms
0.8043024227234753


In [88]:
model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=9.94, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=77,
              reg_alpha=0, reg_lambda=1, ...)

In [89]:
submit=pd.read_csv("/kaggle/input/2022-sju-aichallenge-4-others/submit_sample.csv",index_col=0)
y_test=model.predict(x_test)
submit['Churn']=(y_test)
submit.astype('int64')
submit.to_csv("baseline.csv")